In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import obspy
from IPython.display import Image, display

from utils.misc import print_verification
from utils.plot_all_seismograms import plot_all_seismograms
from utils.plot_ascii_trace import read_ascii_trace

In [ ]:
# Define root_dir only if it is not already defined. This way root_dir stays the same, even if you change the work directory.
try:
    root_dir
except NameError:
    root_dir = Path(os.curdir).absolute()

# SPECFEM bin directory
SPECFEM2D_BIN = root_dir / "specfem2d" / "bin"
# Simulation directory
run_folder = root_dir / "02_Homogeneous_Model"
# Go into run directory
os.chdir(run_folder)
# More readable paths
run_folder = run_folder.relative_to(Path(".").absolute())
output_folder = run_folder / "OUTPUT_FILES"

In [ ]:
def make_output_dir():
    if output_folder.exists():
        shutil.rmtree(output_folder)
    output_folder.mkdir()

# Running the Simulation

In [ ]:
make_output_dir()

Running the mesher

In [ ]:
! $SPECFEM2D_BIN/xmeshfem2D > $output_folder/output_mesher.txt

Running the solver

In [ ]:
! $SPECFEM2D_BIN/xspecfem2D > $output_folder/output_solver.txt

## Checking Stability

In [ ]:
start_pattern = "*** Verification of simulation parameters ***"
end_pattern = "========================================="
started = False
with open(output_folder / "output_solver.txt") as f:
    for line in f:
        if start_pattern in line:
            started = True
        if started and end_pattern in line:
            started = False
            break
        if started:
            print(line.strip())

## Wavefield Plot

In [ ]:
Image(output_folder / "forward_image000000300.jpg")

## Seismograms

In [ ]:
bxx_files = sorted(output_folder.glob("*BXX.semd"))
bxz_files = sorted(output_folder.glob("*BXZ.semd"))
n_stations = len(bxx_files)

fig, axes = plt.subplots(
    nrows=n_stations, ncols=2, figsize=(20, 2 * n_stations), sharex=True, sharey=True
)
for i, bxx_file in enumerate(bxx_files):
    tr = read_ascii_trace(bxx_file)
    axes[i, 0].plot(tr.times() + tr.stats.b, tr.data, "k", label=tr.id)
    axes[i, 0].legend()
    axes[i, 0].set_xlim(tr.stats.b, tr.times()[-1] + tr.stats.b)

for i, bxz_file in enumerate(bxz_files):
    tr = read_ascii_trace(bxz_file)
    axes[i, 1].plot(tr.times() + tr.stats.b, tr.data, "k", label=tr.id)
    axes[i, 1].legend()

fig.tight_layout()

## Changing Source Frequency
Maximum suggested (Ricker) source frequency is `10.88 Hz`. Let's see what happens when we exceed it.

Currently it is `10 Hz`.

In [ ]:
! grep -v ^# DATA/SOURCE

In [ ]:
! ../utils/parfile -f DATA/SOURCE -s f0 30.0

In [ ]:
! grep -v ^# DATA/SOURCE

### Rerun the Simulations

In [ ]:
make_output_dir()
! $SPECFEM2D_BIN/xmeshfem2D > $output_folder/output_mesher.txt
! $SPECFEM2D_BIN/xspecfem2D > $output_folder/output_solver.txt

### Wavefield Plot

In [ ]:
Image(output_folder / "forward_image000000300.jpg")

### Seismograms

In [ ]:
fig, axes = plot_all_seismograms(output_folder, sharey=True)
fig.tight_layout()

### Filtered Seismograms

In [ ]:
fig, axes = plot_all_seismograms(output_folder, bandpass_range=(0.01, 15), sharey=True)
fig.tight_layout()

### Resetting f0

In [ ]:
# Set the default f0
! ../utils/parfile -f DATA/SOURCE -s f0 10.0